In [12]:
import requests
import json
import threading

class WeatherApp:
    def __init__(self):
        self.api_key = "89df9e01aaf466ee40550f3f84852474"
        self.base_url = "http://api.openweathermap.org/data/2.5/weather?"

    def get_weather(self, city):
        try:
            complete_url = f"{self.base_url}q={city}&appid={self.api_key}"
            response = requests.get(complete_url)
            data = response.json()
            temperature_celsius = data["main"]["temp"] - 273.15
            weather_data = {
                "city": city,
                "temperature": temperature_celsius,
                "conditions": data["weather"][0]["description"]
            }
            return weather_data
        except Exception as e:
            print("An error occurred:", e)
            return None

def fetch_weather_for_cities(cities, weather_app):
    for city in cities:
        weather_data = get_weather_with_retry(weather_app, city)
        if weather_data:
            print("Weather report for", city)
            print("Temperature:", weather_data["temperature"], "°C")
            print("Conditions:", weather_data["conditions"])
            print()

def retry_decorator(func):
    def wrapper(*args, **kwargs):
        for _ in range(3):  # Retry up to 3 times
            try:
                result = func(*args, **kwargs)
                return result
            except Exception as e:
                print("Retrying due to error:", e)
        print("Failed after 3 retries.")
        return None
    return wrapper

@retry_decorator
def get_weather_with_retry(weather_app, city):
    return weather_app.get_weather(city)

def main():
    cities = input("Enter cities (separated by commas): ").split(",")
    cities = [city.strip() for city in cities]

    threads = []
    chunk_size = 5  # Adjust the chunk size as needed
    weather_app = WeatherApp()  # Create one instance of WeatherApp to be shared across threads
    for i in range(0, len(cities), chunk_size):
        chunk = cities[i:i+chunk_size]
        thread = threading.Thread(target=fetch_weather_for_cities, args=(chunk, weather_app))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()

Enter cities (separated by commas): delhi, mumbai, surat
Weather report for delhi
Temperature: 40.06 °C
Conditions: haze

Weather report for mumbai
Temperature: 29.99000000000001 °C
Conditions: haze

Weather report for surat
Temperature: 34.99000000000001 °C
Conditions: few clouds

